 Input Data | Output Data |
| --- | --- |
| Region | Tokyo - city/ward, district/town/village |
| Price range (Korean standards) | Accurate price (price) |
| Room Type | Private room, Entire home/apt |
| Accommodates number of guests | Host-written accommodation description (neighborhood_overview) |
| Bathrooms (bathrooms_text) | Host identity verification (host_identity_verified) |
| Bedrooms (bedrooms) | Customer-written accommodation description (description) |
| Beds: number of beds | Minimum number of nights (minimum_nights) |
| Rating (review_scores_rating) range | Overall rating score (review_scores_rating), Number of reviews in the last 30 days (number_of_reviews_l30d), Date of the last review (last_review), Date calendar was last updated (calendar_updated) |

In [67]:
import pandas as pd
import numpy as np
import random

In [68]:
data_file = '/Users/genie/Documents/COLLABORATION/AirbnbWise/Tokyo_Airbnb/data/listings.csv.gz'

listing_gz = pd.read_csv(data_file, compression='gzip', header=0, sep=',', quotechar='"')
listing_gz

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,197677,https://www.airbnb.com/rooms/197677,20230629055629,2023-06-29,city scrape,Rental unit in Sumida · ★4.78 · 1 bedroom · 2 ...,<b>The space</b><br />We are happy to welcome ...,NaN,https://a0.muscache.com/pictures/38437056/d27f...,964081,...,4.83,4.53,4.79,M130003350,f,1,1,0,0,1.21
1,776070,https://www.airbnb.com/rooms/776070,20230629055629,2023-06-29,city scrape,Home in Kita-ku · ★4.98 · 1 bedroom · 1 bed · ...,We have been in airbnb since 2011 and it has g...,We love Nishinippori because is nearer to Toky...,https://a0.muscache.com/pictures/efd9f039-dbd2...,801494,...,4.98,4.83,4.91,NaN,f,1,0,1,0,1.89
2,905944,https://www.airbnb.com/rooms/905944,20230629055629,2023-06-29,city scrape,Rental unit in Shibuya · ★4.76 · 2 bedrooms · ...,NEWLY RENOVATED property entirely for you & yo...,Hatagaya is a great neighborhood located 4 min...,https://a0.muscache.com/pictures/miso/Hosting-...,4847803,...,4.90,4.77,4.77,Hotels and Inns Business Act | 渋谷区保健所長 | 31渋健生...,t,5,5,0,0,1.49
3,1016831,https://www.airbnb.com/rooms/1016831,20230629055629,2023-06-29,city scrape,Home in Setagaya · ★4.94 · 1 bedroom · 2 beds ...,"Hi there, I am Wakana and I live with my two f...",The location is walkable distance to famous Sh...,https://a0.muscache.com/pictures/airflow/Hosti...,5596383,...,4.98,4.92,4.89,NaN,f,1,0,1,0,1.96
4,1196177,https://www.airbnb.com/rooms/1196177,20230629055629,2023-06-29,city scrape,Home in 足立区 · ★4.71 · 1 bedroom · 1.5 shared b...,Ｓtay with host.We can help your travel.<br />B...,There are shopping mall near Senjuohashi stati...,https://a0.muscache.com/pictures/72890882/05ec...,5686404,...,4.88,4.67,4.75,NaN,f,1,0,1,0,0.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11172,922873759575839340,https://www.airbnb.com/rooms/922873759575839340,20230629055629,2023-06-29,city scrape,Rental unit in Shinjuku City · ★New · 1 bedroo...,Higashi Shinjuku station 1 mins on foot.<br />...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,294322772,...,NaN,NaN,NaN,Hotels and Inns Business Act | 新宿区保健所 | 31新保衛環...,t,10,10,0,0,NaN
11173,922998222764343634,https://www.airbnb.com/rooms/922998222764343634,20230629055629,2023-06-29,city scrape,Rental unit in Shinjuku City · ★New · 1 bedroo...,公寓位于城市中心东京新宿区，距离西新宿五丁目步行仅需要7分钟。<br />公寓是三层楼的建筑...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,518343589,...,NaN,NaN,NaN,Hotels and Inns Business Act | 新宿区保健所 | 2新保衛環第94号,f,12,12,0,0,NaN
11174,923008447744339896,https://www.airbnb.com/rooms/923008447744339896,20230629055629,2023-06-29,city scrape,Rental unit in Shinjuku City · ★New · 1 bedroo...,公寓位于城市中心东京新宿区，距离西新宿五丁目步行仅需要7分钟。<br />公寓是三层楼的建筑...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,518343589,...,NaN,NaN,NaN,Hotels and Inns Business Act | 新宿区保健所 | 2新保衛環第94号,f,12,12,0,0,NaN
11175,923011844205437846,https://www.airbnb.com/rooms/923011844205437846,20230629055629,2023-06-29,city scrape,Rental unit in Shinjuku City · ★New · 3 bedroo...,公寓位于城市中心东京新宿区，距离西新宿五丁目步行仅需要7分钟。<br />公寓是三层楼的建筑...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,518343589,...,NaN,NaN,NaN,Hotels and Inns Business Act | 新宿区保健所 | 2新保衛環第94号,f,12,12,0,0,NaN


### User Class

In [69]:
class User:
    def __init__(self, idx, listing_id, host_id, review_scores_rating, bedrooms, beds, bathrooms, region = None, price = None, accommodates = None, minimum_nights = None, id = None, password = None):
        self.idx = idx
        self.listing_id = listing_id
        self.host_id = host_id
        self.review_scores_rating = review_scores_rating
        self.bedrooms = bedrooms
        self.beds = beds
        self.bathrooms = bathrooms
        self.region = region
        self.price = price
        self.accommodates = accommodates
        self.minimum_nights = minimum_nights
        

In [70]:
import random

sample = listing_gz.loc[:, [ 'id', 'host_id', 'review_scores_rating', 'bedrooms', 'beds', 'bathrooms_text', 'neighbourhood_cleansed', 'price', 'accommodates', 'minimum_nights']]
sample.reset_index(drop=True, inplace=True)
sample.sort_values(by='review_scores_rating', ascending=False, inplace=True)
sample = sample.loc[:, :]
sample.reset_index(drop=True, inplace=True)
sample


,id,host_id,review_scores_rating,bedrooms,beds,bathrooms_text,neighbourhood_cleansed,price,accommodates,minimum_nights
0,922833637376956356,490272024,5.0,1.0,1.0,1 bath,Koto Ku,"$9,600.00",2,1
1,727140790042920016,481595902,5.0,1.0,2.0,1 bath,Sumida Ku,"$11,536.00",4,2
2,729355347138793951,481729050,5.0,1.0,1.0,1 shared bath,Shinjuku Ku,"$21,600.00",3,1
3,30049203,68181338,5.0,1.0,6.0,1 private bath,Arakawa Ku,"$25,000.00",7,1
4,30047622,68181338,5.0,1.0,2.0,0 shared baths,Arakawa Ku,"$9,000.00",2,1
...,...,...,...,...,...,...,...,...,...,...
11172,922873759575839340,294322772,NaN,1.0,3.0,1 bath,Shinjuku Ku,"$12,000.00",4,2
11173,922998222764343634,518343589,NaN,1.0,2.0,1 bath,Shinjuku Ku,"$16,000.00",3,2
11174,923008447744339896,518343589,NaN,1.0,2.0,1 bath,Shinjuku Ku,"$16,000.00",4,2
11175,923011844205437846,518343589,NaN,3.0,6.0,1 bath,Shinjuku Ku,"$40,000.00",9,2


In [71]:
import csv 

user_list = []

for i in range(len(sample)):
    #price = sample['price'][i].replace('$', '').replace(',', '')
    user = User(idx = i, listing_id = sample['id'][i], host_id = sample['host_id'][i], review_scores_rating = sample['review_scores_rating'][i], bedrooms = sample['bedrooms'][i], beds = sample['beds'][i], bathrooms = sample['bathrooms_text'][i],region = sample['neighbourhood_cleansed'][i], price = sample['price'][i], accommodates = sample['accommodates'][i], minimum_nights = sample['minimum_nights'][i])
    user_list.append(user)

# CSV 파일에 데이터 쓰기
csv_filename = '/Users/genie/Documents/COLLABORATION/AirbnbWise/Pipeline/csv/user_totaldb.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = ['idx', 'listing_id', 'host_id', 'review_scores_rating', 'bedrooms', 'beds', 'bathrooms','region', 'price', 'accommodates', 'minimum_nights']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # CSV 파일의 헤더 작성
    writer.writeheader()
    
    # 사용자 데이터를 CSV 파일에 작성
    for user in user_list:
        writer.writerow({
            'idx' : user.idx,
            'listing_id' : user.listing_id,
            'host_id' : user.host_id,
            'review_scores_rating' : user.review_scores_rating,
            'bedrooms' : user.bedrooms,
            'beds' : user.beds,
            'bathrooms' : user.bathrooms,
            'region': user.region,
            'price': user.price,
            'accommodates': user.accommodates,
            'minimum_nights': user.minimum_nights,
        })

print(f'사용자 데이터가 {csv_filename} 파일에 성공적으로 저장되었습니다.')

사용자 데이터가 /Users/genie/Documents/COLLABORATION/AirbnbWise/Pipeline/csv/user_totaldb.csv 파일에 성공적으로 저장되었습니다.


In [72]:
user_db = pd.read_csv('/Users/genie/Documents/COLLABORATION/AirbnbWise/Pipeline/csv/user_totaldb.csv')
user_db

#* id는 하나만 있으므로 평점 높은 숙소 대상으로 정렬

,idx,listing_id,host_id,review_scores_rating,bedrooms,beds,bathrooms,region,price,accommodates,minimum_nights
0,0,922833637376956356,490272024,5.0,1.0,1.0,1 bath,Koto Ku,"$9,600.00",2,1
1,1,727140790042920016,481595902,5.0,1.0,2.0,1 bath,Sumida Ku,"$11,536.00",4,2
2,2,729355347138793951,481729050,5.0,1.0,1.0,1 shared bath,Shinjuku Ku,"$21,600.00",3,1
3,3,30049203,68181338,5.0,1.0,6.0,1 private bath,Arakawa Ku,"$25,000.00",7,1
4,4,30047622,68181338,5.0,1.0,2.0,0 shared baths,Arakawa Ku,"$9,000.00",2,1
...,...,...,...,...,...,...,...,...,...,...,...
11172,11172,922873759575839340,294322772,NaN,1.0,3.0,1 bath,Shinjuku Ku,"$12,000.00",4,2
11173,11173,922998222764343634,518343589,NaN,1.0,2.0,1 bath,Shinjuku Ku,"$16,000.00",3,2
11174,11174,923008447744339896,518343589,NaN,1.0,2.0,1 bath,Shinjuku Ku,"$16,000.00",4,2
11175,11175,923011844205437846,518343589,NaN,3.0,6.0,1 bath,Shinjuku Ku,"$40,000.00",9,2
